In [1]:
import polars as pl
import numpy as np
import matplotlib.pyplot as plt

from catboost import Pool, CatBoostClassifier
from sklearn.metrics import roc_auc_score

In [2]:
data_path = '/media/dm/1TB/Data Fusion Contest 2026/task2/'

In [3]:
%%time
#train = pl.read_parquet('data/train_main_features.parquet')
#test = pl.read_parquet('data/test_main_features.parquet')

train = pl.read_parquet(data_path+'train_main_features.parquet'
                       ).fill_null(-99)
test = pl.read_parquet(data_path+'test_main_features.parquet'
                      ).fill_null(-99)

print('Тренировочные данные:', train.shape)
print('Тестовые данные:', test.shape)
train

Тренировочные данные: (750000, 200)
Тестовые данные: (250000, 200)
CPU times: user 1.1 s, sys: 2.03 s, total: 3.13 s
Wall time: 709 ms


customer_id,cat_feature_1,cat_feature_2,cat_feature_3,cat_feature_4,cat_feature_5,cat_feature_6,cat_feature_7,cat_feature_8,cat_feature_9,cat_feature_10,cat_feature_11,cat_feature_12,cat_feature_13,cat_feature_14,cat_feature_15,cat_feature_16,cat_feature_17,cat_feature_18,cat_feature_19,cat_feature_20,cat_feature_21,cat_feature_22,cat_feature_23,cat_feature_24,cat_feature_25,cat_feature_26,cat_feature_27,cat_feature_28,cat_feature_29,cat_feature_30,cat_feature_31,cat_feature_32,cat_feature_33,cat_feature_34,cat_feature_35,cat_feature_36,…,num_feature_96,num_feature_97,num_feature_98,num_feature_99,num_feature_100,num_feature_101,num_feature_102,num_feature_103,num_feature_104,num_feature_105,num_feature_106,num_feature_107,num_feature_108,num_feature_109,num_feature_110,num_feature_111,num_feature_112,num_feature_113,num_feature_114,num_feature_115,num_feature_116,num_feature_117,num_feature_118,num_feature_119,num_feature_120,num_feature_121,num_feature_122,num_feature_123,num_feature_124,num_feature_125,num_feature_126,num_feature_127,num_feature_128,num_feature_129,num_feature_130,num_feature_131,num_feature_132
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1000001,1.0,0.0,2.0,1.0,2.0,3.0,2.0,2.0,4.0,2.0,0.0,0.0,0.0,1.0,2.0,0.0,2.0,2.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,2.0,1.0,2.0,2.0,0.0,0.0,212.0,0.0,0.0,…,-0.284519,-99.0,-0.004499,-99.0,-0.050159,-0.002297,-0.043592,-0.061488,0.450342,-99.0,0.0,-99.0,-0.060492,-0.279105,-99.0,-0.429813,-99.0,-0.009654,-0.293036,-99.0,-0.493959,-0.019079,-99.0,-99.0,-99.0,-99.0,-0.001357,-0.031281,-0.046146,-99.0,-0.445279,-99.0,-99.0,-0.107666,-0.418616,-99.0,-99.0
1000002,1.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,212.0,1.0,0.0,…,-0.284519,0.460275,-0.004278,-99.0,-0.556244,-0.002297,-0.043592,-0.061488,1.998252,-0.009552,0.0,-0.005762,-0.060492,-0.279105,-99.0,-0.429813,-99.0,-0.009654,-0.293036,-0.004421,-0.256445,-0.014154,-99.0,-0.24167,-99.0,-99.0,-0.001357,-0.031281,-0.046146,-0.10217,1.550722,-99.0,-99.0,-0.170724,-0.805771,-0.397803,-0.373734
1000003,1.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,212.0,0.0,0.0,…,-0.284519,-0.264397,-0.004278,-99.0,-99.0,-0.002297,-0.043592,-0.061488,-0.264078,-99.0,0.0,-0.26649,-0.060492,-0.279105,-99.0,-0.429813,-99.0,-0.009654,-0.293036,-0.004421,-0.57313,-0.019124,-99.0,-0.24167,-99.0,-99.0,-0.001357,-0.031281,-0.046146,-99.0,-0.475778,-99.0,-99.0,-0.170724,-0.602005,-0.397803,-0.373734
1000004,1.0,0.0,2.0,1.0,2.0,3.0,2.0,2.0,3.0,2.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,2.0,1.0,2.0,2.0,0.0,0.0,212.0,0.0,0.0,…,-0.284519,-99.0,-0.004499,-99.0,-99.0,-0.002297,-0.043592,-0.061488,0.688482,-0.009552,0.0,-99.0,-0.060492,-99.0,-99.0,-0.429813,-99.0,-0.009654,-0.293036,-99.0,-0.57313,-99.0,-99.0,-0.505441,-99.0,0.714631,-0.001357,-0.031281,-0.046146,-99.0,-0.475778,0.111196,0.116695,-99.0,-0.724265,-99.0,-99.0
1000005,1.0,2.0,0.0,1.0,0.0,3.0,0.0,0.0,2.0,1.0,2.0,2.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,0.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,2.0,2.0,212.0,2.0,2.0,…,-0.284519,-0.264397,-0.004499,-99.0,-99.0,-0.002297,-0.043592,-99.0,-0.264078,-99.0,0.0,-0.26649,-99.0,-0.279105,-99.0,-0.429813,-99.0,-0.009654,-0.293036,-0.004421,-0.57313,-0.018674,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-0.046146,-99.0,-99.0,-99.0,-99.0,-0.107666,-99.0,-0.397803,-0.373734
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1749996,1.0,0.0,2.0,0.0,2.0,3.0,2.0,2.0,4.0,2.0,0.0,0.0,0.0,1.0,2.0,0.0,2.0,2.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0

In [4]:
missing_values = (
    train.null_count()
    .unpivot(variable_name="Column", value_name="Missing Values")
    .filter(pl.col("Missing Values") > 0)
    .sort("Missing Values", descending=True)
)

print('\nКоличество пропусков в признаках:\n', missing_values)


Количество пропусков в признаках:
 shape: (0, 2)
┌────────┬────────────────┐
│ Column ┆ Missing Values │
│ ---    ┆ ---            │
│ str    ┆ u64            │
╞════════╪════════════════╡
└────────┴────────────────┘


In [5]:
cat_feature_names = [
    col_name for col_name in train.columns 
    if col_name.startswith("cat_feature")
]
cat_feature_names

['cat_feature_1',
 'cat_feature_2',
 'cat_feature_3',
 'cat_feature_4',
 'cat_feature_5',
 'cat_feature_6',
 'cat_feature_7',
 'cat_feature_8',
 'cat_feature_9',
 'cat_feature_10',
 'cat_feature_11',
 'cat_feature_12',
 'cat_feature_13',
 'cat_feature_14',
 'cat_feature_15',
 'cat_feature_16',
 'cat_feature_17',
 'cat_feature_18',
 'cat_feature_19',
 'cat_feature_20',
 'cat_feature_21',
 'cat_feature_22',
 'cat_feature_23',
 'cat_feature_24',
 'cat_feature_25',
 'cat_feature_26',
 'cat_feature_27',
 'cat_feature_28',
 'cat_feature_29',
 'cat_feature_30',
 'cat_feature_31',
 'cat_feature_32',
 'cat_feature_33',
 'cat_feature_34',
 'cat_feature_35',
 'cat_feature_36',
 'cat_feature_37',
 'cat_feature_38',
 'cat_feature_39',
 'cat_feature_40',
 'cat_feature_41',
 'cat_feature_42',
 'cat_feature_43',
 'cat_feature_44',
 'cat_feature_45',
 'cat_feature_46',
 'cat_feature_47',
 'cat_feature_48',
 'cat_feature_49',
 'cat_feature_50',
 'cat_feature_51',
 'cat_feature_52',
 'cat_feature_53',
 '

In [6]:
train = train.with_columns(
    pl.col(cat_feature_names).cast(pl.Int32)
)
train 

customer_id,cat_feature_1,cat_feature_2,cat_feature_3,cat_feature_4,cat_feature_5,cat_feature_6,cat_feature_7,cat_feature_8,cat_feature_9,cat_feature_10,cat_feature_11,cat_feature_12,cat_feature_13,cat_feature_14,cat_feature_15,cat_feature_16,cat_feature_17,cat_feature_18,cat_feature_19,cat_feature_20,cat_feature_21,cat_feature_22,cat_feature_23,cat_feature_24,cat_feature_25,cat_feature_26,cat_feature_27,cat_feature_28,cat_feature_29,cat_feature_30,cat_feature_31,cat_feature_32,cat_feature_33,cat_feature_34,cat_feature_35,cat_feature_36,…,num_feature_96,num_feature_97,num_feature_98,num_feature_99,num_feature_100,num_feature_101,num_feature_102,num_feature_103,num_feature_104,num_feature_105,num_feature_106,num_feature_107,num_feature_108,num_feature_109,num_feature_110,num_feature_111,num_feature_112,num_feature_113,num_feature_114,num_feature_115,num_feature_116,num_feature_117,num_feature_118,num_feature_119,num_feature_120,num_feature_121,num_feature_122,num_feature_123,num_feature_124,num_feature_125,num_feature_126,num_feature_127,num_feature_128,num_feature_129,num_feature_130,num_feature_131,num_feature_132
i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1000001,1,0,2,1,2,3,2,2,4,2,0,0,0,1,2,0,2,2,2,0,0,0,2,0,0,0,1,2,1,2,2,0,0,212,0,0,…,-0.284519,-99.0,-0.004499,-99.0,-0.050159,-0.002297,-0.043592,-0.061488,0.450342,-99.0,0.0,-99.0,-0.060492,-0.279105,-99.0,-0.429813,-99.0,-0.009654,-0.293036,-99.0,-0.493959,-0.019079,-99.0,-99.0,-99.0,-99.0,-0.001357,-0.031281,-0.046146,-99.0,-0.445279,-99.0,-99.0,-0.107666,-0.418616,-99.0,-99.0
1000002,1,0,0,1,0,3,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,212,1,0,…,-0.284519,0.460275,-0.004278,-99.0,-0.556244,-0.002297,-0.043592,-0.061488,1.998252,-0.009552,0.0,-0.005762,-0.060492,-0.279105,-99.0,-0.429813,-99.0,-0.009654,-0.293036,-0.004421,-0.256445,-0.014154,-99.0,-0.24167,-99.0,-99.0,-0.001357,-0.031281,-0.046146,-0.10217,1.550722,-99.0,-99.0,-0.170724,-0.805771,-0.397803,-0.373734
1000003,1,0,0,1,0,3,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,1,0,0,0,0,0,0,212,0,0,…,-0.284519,-0.264397,-0.004278,-99.0,-99.0,-0.002297,-0.043592,-0.061488,-0.264078,-99.0,0.0,-0.26649,-0.060492,-0.279105,-99.0,-0.429813,-99.0,-0.009654,-0.293036,-0.004421,-0.57313,-0.019124,-99.0,-0.24167,-99.0,-99.0,-0.001357,-0.031281,-0.046146,-99.0,-0.475778,-99.0,-99.0,-0.170724,-0.602005,-0.397803,-0.373734
1000004,1,0,2,1,2,3,2,2,3,2,0,0,0,1,2,2,2,2,2,0,0,2,2,0,0,0,0,2,1,2,2,0,0,212,0,0,…,-0.284519,-99.0,-0.004499,-99.0,-99.0,-0.002297,-0.043592,-0.061488,0.688482,-0.009552,0.0,-99.0,-0.060492,-99.0,-99.0,-0.429813,-99.0,-0.009654,-0.293036,-99.0,-0.57313,-99.0,-99.0,-0.505441,-99.0,0.714631,-0.001357,-0.031281,-0.046146,-99.0,-0.475778,0.111196,0.116695,-99.0,-0.724265,-99.0,-99.0
1000005,1,2,0,1,0,3,0,0,2,1,2,2,2,0,1,0,0,0,0,2,2,2,0,2,2,2,2,0,0,0,0,2,2,212,2,2,…,-0.284519,-0.264397,-0.004499,-99.0,-99.0,-0.002297,-0.043592,-99.0,-0.264078,-99.0,0.0,-0.26649,-99.0,-0.279105,-99.0,-0.429813,-99.0,-0.009654,-0.293036,-0.004421,-0.57313,-0.018674,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-0.046146,-99.0,-99.0,-99.0,-99.0,-0.107666,-99.0,-0.397803,-0.373734
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1749996,1,0,2,0,2,3,2,2,4,2,0,0,0,1,2,0,2,2,2,0,0,2,2,0,0,0,0,2,1,2,2,0,0,212,0,0,…,-99.0,-99.0,-0.004499,-0.150243,-99.0,-99.0,-99.0,2.831928,-99.0,-99.0,-99.0,-99.0,-0.060492,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-0.019236,-99.0,-0.505441,-99.0,-99.0,-0.001357,-0.031281,-99.0,-99.0,-0.475778,-99.0,-99.0,-99.0,0.335315,-99.0,-99.0
1749997,1,0,2,1,2,3,2,2,2,2,0,0,0,1,2,0,2,2,2,0,0,0,2,0,0,0,0,2,1,2,2,0,0,212,1,0,…,-0.284519,-99.0,-0.004499,-99.0,-0.5100

In [7]:
target = pl.read_parquet(data_path+'train_target.parquet')
target
#shape: (750_000, 42)

customer_id,target_1_1,target_1_2,target_1_3,target_1_4,target_1_5,target_2_1,target_2_2,target_2_3,target_2_4,target_2_5,target_2_6,target_2_7,target_2_8,target_3_1,target_3_2,target_3_3,target_3_4,target_3_5,target_4_1,target_5_1,target_5_2,target_6_1,target_6_2,target_6_3,target_6_4,target_6_5,target_7_1,target_7_2,target_7_3,target_8_1,target_8_2,target_8_3,target_9_1,target_9_2,target_9_3,target_9_4,target_9_5,target_9_6,target_9_7,target_9_8,target_10_1
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1000001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1749996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1749997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1749998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [8]:
target_columns = [col for col in target.columns if col.startswith("target")]
target_columns

['target_1_1',
 'target_1_2',
 'target_1_3',
 'target_1_4',
 'target_1_5',
 'target_2_1',
 'target_2_2',
 'target_2_3',
 'target_2_4',
 'target_2_5',
 'target_2_6',
 'target_2_7',
 'target_2_8',
 'target_3_1',
 'target_3_2',
 'target_3_3',
 'target_3_4',
 'target_3_5',
 'target_4_1',
 'target_5_1',
 'target_5_2',
 'target_6_1',
 'target_6_2',
 'target_6_3',
 'target_6_4',
 'target_6_5',
 'target_7_1',
 'target_7_2',
 'target_7_3',
 'target_8_1',
 'target_8_2',
 'target_8_3',
 'target_9_1',
 'target_9_2',
 'target_9_3',
 'target_9_4',
 'target_9_5',
 'target_9_6',
 'target_9_7',
 'target_9_8',
 'target_10_1']

In [9]:
model = CatBoostClassifier(
                           #one_hot_max_size=1024*2,
                           one_hot_max_size=256,
                           
                           loss_function = 'Logloss', 
                           custom_metric=["Accuracy","AUC"],
                           eval_metric="AUC",
                           nan_mode = 'Min', 
                           random_seed = 1234,
    
                           task_type="GPU",
                           verbose = 100)

In [10]:
%%time
limit=500000
limit=700000
for tcols in target_columns:
    print(tcols)
    train_pool = Pool(data = train[:limit].drop("customer_id").to_pandas(), 
                      label = target[:limit].select(tcols
                                                   #).drop("customer_id"
                                                         ).to_pandas(), 
                      #cat_features = cat_feature_names
                     )

    valid_pool = Pool(data = train[limit:].drop("customer_id").to_pandas(), 
                      label = target[limit:].select(tcols
                                                   #).drop("customer_id"
                                                         ).to_pandas(), 
                      #cat_features = cat_feature_names
                     )
    
    model.fit(train_pool,
              plot=True,
              eval_set=valid_pool,
             )
    model.save_model(f"wocat{tcols}.cbm")

target_1_1


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.040803


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.7024572	best: 0.7024572 (0)	total: 115ms	remaining: 1m 54s
100:	test: 0.8921201	best: 0.8921201 (100)	total: 1.56s	remaining: 13.9s
200:	test: 0.8998920	best: 0.8998920 (200)	total: 2.87s	remaining: 11.4s
300:	test: 0.9026652	best: 0.9026652 (300)	total: 4.13s	remaining: 9.59s
400:	test: 0.9047348	best: 0.9047348 (400)	total: 5.35s	remaining: 7.99s
500:	test: 0.9062008	best: 0.9062008 (500)	total: 6.58s	remaining: 6.55s
600:	test: 0.9078758	best: 0.9078758 (600)	total: 7.8s	remaining: 5.18s
700:	test: 0.9089053	best: 0.9089053 (700)	total: 9.01s	remaining: 3.84s
800:	test: 0.9096825	best: 0.9096825 (800)	total: 10.2s	remaining: 2.54s
900:	test: 0.9103254	best: 0.9103254 (900)	total: 11.4s	remaining: 1.26s
999:	test: 0.9111404	best: 0.9111404 (999)	total: 12.6s	remaining: 0us
bestTest = 0.9111403525
bestIteration = 999
target_1_2


...

100:	test: 0.9019813	best: 0.9019813 (100)	total: 1.41s	remaining: 12.5s
200:	test: 0.9071426	best: 0.9071426 (200)	total: 2.81s	remaining: 11.2s
300:	test: 0.9101800	best: 0.9101800 (300)	total: 4.08s	remaining: 9.48s
400:	test: 0.9123516	best: 0.9123516 (400)	total: 5.3s	remaining: 7.92s
500:	test: 0.9136121	best: 0.9136121 (500)	total: 6.53s	remaining: 6.5s
600:	test: 0.9146412	best: 0.9146412 (600)	total: 7.77s	remaining: 5.16s
700:	test: 0.9153526	best: 0.9153526 (700)	total: 8.99s	remaining: 3.83s
800:	test: 0.9158593	best: 0.9158593 (800)	total: 10.2s	remaining: 2.54s
900:	test: 0.9163943	best: 0.9163943 (900)	total: 11.6s	remaining: 1.28s
999:	test: 0.9167934	best: 0.9168020 (995)	total: 12.9s	remaining: 0us
bestTest = 0.9168020487
bestIteration = 995
Shrink model to first 996 iterations.
target_10_1


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.040803
0:	test: 0.7029632	best: 0.7029632 (0)	total: 17.8ms	remaining: 17.8s


Default metric period is 5 because AUC is/are not implemented for GPU


100:	test: 0.7310009	best: 0.7310009 (100)	total: 1.54s	remaining: 13.7s
200:	test: 0.7359317	best: 0.7359317 (200)	total: 2.97s	remaining: 11.8s
300:	test: 0.7385590	best: 0.7385590 (300)	total: 4.42s	remaining: 10.3s
400:	test: 0.7404480	best: 0.7404480 (400)	total: 5.82s	remaining: 8.69s
500:	test: 0.7417484	best: 0.7417484 (500)	total: 7.21s	remaining: 7.18s
600:	test: 0.7427355	best: 0.7427355 (600)	total: 8.61s	remaining: 5.72s
700:	test: 0.7434476	best: 0.7434476 (700)	total: 10s	remaining: 4.28s
800:	test: 0.7440533	best: 0.7440533 (800)	total: 11.4s	remaining: 2.84s
900:	test: 0.7444533	best: 0.7444533 (900)	total: 12.8s	remaining: 1.41s
999:	test: 0.7449418	best: 0.7449418 (999)	total: 14.2s	remaining: 0us
bestTest = 0.7449418306
bestIteration = 999
CPU times: user 14min 14s, sys: 6min 44s, total: 20min 59s
Wall time: 9min 41s


In [11]:
test = pl.read_parquet(data_path+'test_main_features.parquet'
                      ).fill_null(-99)

test = test.with_columns(
    pl.col(cat_feature_names).cast(pl.Int32)
)
test

customer_id,cat_feature_1,cat_feature_2,cat_feature_3,cat_feature_4,cat_feature_5,cat_feature_6,cat_feature_7,cat_feature_8,cat_feature_9,cat_feature_10,cat_feature_11,cat_feature_12,cat_feature_13,cat_feature_14,cat_feature_15,cat_feature_16,cat_feature_17,cat_feature_18,cat_feature_19,cat_feature_20,cat_feature_21,cat_feature_22,cat_feature_23,cat_feature_24,cat_feature_25,cat_feature_26,cat_feature_27,cat_feature_28,cat_feature_29,cat_feature_30,cat_feature_31,cat_feature_32,cat_feature_33,cat_feature_34,cat_feature_35,cat_feature_36,…,num_feature_96,num_feature_97,num_feature_98,num_feature_99,num_feature_100,num_feature_101,num_feature_102,num_feature_103,num_feature_104,num_feature_105,num_feature_106,num_feature_107,num_feature_108,num_feature_109,num_feature_110,num_feature_111,num_feature_112,num_feature_113,num_feature_114,num_feature_115,num_feature_116,num_feature_117,num_feature_118,num_feature_119,num_feature_120,num_feature_121,num_feature_122,num_feature_123,num_feature_124,num_feature_125,num_feature_126,num_feature_127,num_feature_128,num_feature_129,num_feature_130,num_feature_131,num_feature_132
i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1750001,1,0,2,0,2,3,2,2,4,0,0,0,0,1,0,0,2,2,2,0,0,0,2,0,0,0,1,2,1,2,2,0,0,212,0,0,…,0.503368,-0.264397,-0.004499,-0.150243,-0.241966,-0.002297,-0.043592,-0.061488,-0.383148,-0.009552,0.0,-0.26649,-0.060492,-0.279105,-99.0,-0.429813,-99.0,-0.009654,-0.293036,-99.0,-0.493959,-0.01556,-99.0,-99.0,-99.0,-99.0,-0.001357,-0.031281,-0.046146,0.062409,-0.475778,-99.0,-99.0,0.018451,-0.765018,-0.397803,-0.373734
1750002,0,0,2,0,2,3,2,2,4,0,0,0,0,1,0,0,2,2,2,1,0,0,2,1,0,0,1,2,1,2,2,1,0,212,1,0,…,-0.284519,-0.264397,-0.004278,0.386748,-0.620003,-0.002297,-0.043592,0.367167,-0.025938,-99.0,0.0,-0.26649,-0.060492,-0.279105,-99.0,-0.429813,-99.0,-0.009654,3.569151,-99.0,-0.57313,-0.015727,-99.0,-99.0,-0.3427,-99.0,-0.001357,-0.031281,-0.046146,-99.0,-0.475778,-99.0,-99.0,-0.170724,-0.357487,-0.397803,-0.373734
1750003,1,0,2,1,2,3,2,2,4,2,0,0,0,1,2,0,2,2,2,0,0,0,2,0,0,0,1,2,1,2,2,1,0,212,0,0,…,-99.0,-0.264397,-0.004499,-99.0,-99.0,-0.002297,-99.0,-0.061488,-0.264078,-0.009468,-99.0,-0.26649,-0.060492,-0.279105,-99.0,-99.0,-99.0,-0.009654,-99.0,-99.0,-99.0,-0.01934,-99.0,0.022101,-99.0,-99.0,-99.0,-0.031281,-99.0,-99.0,-0.455446,-99.0,-99.0,0.018451,-0.785394,-99.0,-99.0
1750004,1,0,2,0,2,3,2,2,4,2,0,0,0,1,2,0,2,2,2,0,0,2,2,0,0,0,1,2,1,2,2,0,0,212,0,0,…,-0.284519,-99.0,-0.004499,-0.150243,-99.0,-0.002297,-0.043592,-0.061488,-0.383148,-99.0,0.0,-99.0,-0.060492,-99.0,-99.0,-0.429813,-99.0,-0.009654,-0.293036,-99.0,-0.018932,-0.015257,-99.0,-99.0,-0.388464,-99.0,-0.001357,-0.031281,-0.046146,0.054572,-0.475778,-99.0,-99.0,-99.0,-0.520499,-99.0,-99.0
1750005,0,1,2,0,2,3,2,2,4,2,0,0,0,1,2,0,2,2,2,1,0,0,2,1,1,0,1,2,1,2,2,1,0,212,0,0,…,-0.284519,-0.264397,-0.004499,-0.150243,1.663358,-0.002297,-0.043592,-0.061488,-0.383148,-99.0,0.0,-0.26649,-0.060492,-0.279105,-99.0,-0.429813,-99.0,-0.009654,-0.293036,-99.0,-0.57313,-0.017804,-99.0,-0.24167,-99.0,-0.138239,-0.001357,-0.031281,-0.046146,-99.0,1.533778,1.318607,-99.0,-0.170724,0.314939,-99.0,-99.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1999996,1,2,2,1,2,3,2,2,4,2,2,2,2,1,2,0,2,2,2,2,2,2,2,2,2,2,2,2,1,2,2,2,2,212,2,2,…,-99.0,-99.0,-0.004499,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
1999997,1,2,0,1,0,3,0,0,4,1,2,2,2,0,0,0,0,0,0,2,2,0,0,2,2,2,2,0,0,0,0,2,2,212,2,2,…,-0.284519,-0.264397,-0.004499,-99.0,-0.761335,-0.002297,-0.043592,-

In [12]:
test_pool = Pool(data = test.drop("customer_id").to_pandas(), 
                 #cat_features = cat_feature_names
                )
test_pool

In [13]:
for tcols in target_columns:
    print(tcols)
    model.load_model(f"wocat{tcols}.cbm")
    test_predict = model.predict(test_pool, prediction_type = "RawFormulaVal")
    test = test.with_columns(temp=test_predict).rename({"temp":tcols})
test

target_1_1
target_1_2
target_1_3
target_1_4
target_1_5
target_2_1
target_2_2
target_2_3
target_2_4
target_2_5
target_2_6
target_2_7
target_2_8
target_3_1
target_3_2
target_3_3
target_3_4
target_3_5
target_4_1
target_5_1
target_5_2
target_6_1
target_6_2
target_6_3
target_6_4
target_6_5
target_7_1
target_7_2
target_7_3
target_8_1
target_8_2
target_8_3
target_9_1
target_9_2
target_9_3
target_9_4
target_9_5
target_9_6
target_9_7
target_9_8
target_10_1


customer_id,cat_feature_1,cat_feature_2,cat_feature_3,cat_feature_4,cat_feature_5,cat_feature_6,cat_feature_7,cat_feature_8,cat_feature_9,cat_feature_10,cat_feature_11,cat_feature_12,cat_feature_13,cat_feature_14,cat_feature_15,cat_feature_16,cat_feature_17,cat_feature_18,cat_feature_19,cat_feature_20,cat_feature_21,cat_feature_22,cat_feature_23,cat_feature_24,cat_feature_25,cat_feature_26,cat_feature_27,cat_feature_28,cat_feature_29,cat_feature_30,cat_feature_31,cat_feature_32,cat_feature_33,cat_feature_34,cat_feature_35,cat_feature_36,…,target_1_5,target_2_1,target_2_2,target_2_3,target_2_4,target_2_5,target_2_6,target_2_7,target_2_8,target_3_1,target_3_2,target_3_3,target_3_4,target_3_5,target_4_1,target_5_1,target_5_2,target_6_1,target_6_2,target_6_3,target_6_4,target_6_5,target_7_1,target_7_2,target_7_3,target_8_1,target_8_2,target_8_3,target_9_1,target_9_2,target_9_3,target_9_4,target_9_5,target_9_6,target_9_7,target_9_8,target_10_1
i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1750001,1,0,2,0,2,3,2,2,4,0,0,0,0,1,0,0,2,2,2,0,0,0,2,0,0,0,1,2,1,2,2,0,0,212,0,0,…,-6.918528,-4.66216,-4.915423,-7.500583,-4.715835,-6.640036,-5.824955,-8.341255,-12.819076,-2.406226,-4.442045,-7.582067,-8.373887,-9.577566,-4.268574,-5.467973,-6.241443,-5.710987,-5.654677,-5.574775,-6.353785,-8.725774,-2.789667,-4.500851,-5.520776,-6.09348,-3.923785,-3.675133,-5.381188,-2.626255,-3.895903,-8.207819,-6.253912,-0.500078,-2.286177,-6.150739,-0.498483
1750002,0,0,2,0,2,3,2,2,4,0,0,0,0,1,0,0,2,2,2,1,0,0,2,1,0,0,1,2,1,2,2,1,0,212,1,0,…,-6.761028,-3.426863,-4.620086,-7.228794,-4.414074,-5.883624,-6.262423,-8.047687,-13.991214,-2.650402,-5.039015,-7.017035,-7.936301,-9.204812,-6.605817,-4.984972,-6.488889,-5.417718,-5.049805,-5.960865,-5.943228,-8.973015,-3.166242,-4.139491,-5.981587,-4.470534,-2.48613,-4.898022,-6.251198,-1.267649,-3.525211,-4.009518,-4.130578,-0.718358,-1.987024,-8.397829,-1.131405
1750003,1,0,2,1,2,3,2,2,4,2,0,0,0,1,2,0,2,2,2,0,0,0,2,0,0,0,1,2,1,2,2,1,0,212,0,0,…,-6.591024,-5.54841,-5.551536,-7.847321,-5.347166,-6.326354,-5.382605,-10.284917,-16.017017,-2.26323,-4.321121,-7.585054,-7.769879,-10.078516,-3.691358,-5.281967,-6.999967,-4.962929,-5.92573,-5.928427,-5.588191,-8.952788,-5.856121,-4.425495,-5.938095,-3.72643,-3.482952,-4.293971,-5.503377,-3.510869,-4.390942,-9.037441,-7.441411,-1.320508,-3.310003,-7.026916,-0.595101
1750004,1,0,2,0,2,3,2,2,4,2,0,0,0,1,2,0,2,2,2,0,0,2,2,0,0,0,1,2,1,2,2,0,0,212,0,0,…,-9.24239,-5.283117,-5.505198,-7.857042,-4.52406,-7.64873,-6.237172,-9.69699,-14.862511,-2.78275,-4.386976,-7.380786,-8.317619,-9.694301,-6.290978,-5.726462,-6.70731,-5.618042,-5.417733,-5.570696,-6.625319,-10.07075,-2.760924,-4.432539,-5.826955,-6.037653,-4.756296,-3.79735,-6.490259,-3.329884,-3.747324,-7.305758,-5.467099,-0.534861,-2.021774,-5.965953,-0.519389
1750005,0,1,2,0,2,3,2,2,4,2,0,0,0,1,2,0,2,2,2,1,0,0,2,1,1,0,1,2,1,2,2,1,0,212,0,0,…,-8.346423,-4.962579,-5.654517,-6.90916,-5.646854,-5.905872,-5.672499,-10.104343,-15.64425,-2.111308,-4.907119,-6.97267,-8.461734,-9.847859,-6.821068,-5.796301,-6.434449,-4.65988,-4.64246,-6.914616,-6.316195,-9.454053,-6.177919,-4.735089,-6.658656,-5.247823,-3.242404,-4.789762,-6.028573,-2.765586,-3.652401,-8.125484,-4.593967,-1.246197,-2.16552,-6.417112,-0.724723
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1999996,1,2,2,1,2,3,2,2,4,2,2,2,2,1,2,0,2,2,2,2,2,2,2,2,2,2,2,2,1,2,2,2,2,212,2,2,…,-8.020615,-7.952994,-6.494458,-9.104302,-4.9239,-6.970297,-5.264845,-12.065701,-16.430234,-3.037751,-4.802828,-7.886857,-8.732921,-9.66837,-7.306258,-5.824976,-6.809601,-4.915778,-5.054389,-5.526528,-5.955274,-10.189803,-3.323383,-4.275666,-6.525299,-5.2948

In [14]:
predict_schema = ["customer_id"]+[col.replace("target_", "predict_") for col in target.columns if col.startswith("target_")]

catboost_predictions = pl.DataFrame(test.select(target.columns), schema = predict_schema)
catboost_predictions

customer_id,predict_1_1,predict_1_2,predict_1_3,predict_1_4,predict_1_5,predict_2_1,predict_2_2,predict_2_3,predict_2_4,predict_2_5,predict_2_6,predict_2_7,predict_2_8,predict_3_1,predict_3_2,predict_3_3,predict_3_4,predict_3_5,predict_4_1,predict_5_1,predict_5_2,predict_6_1,predict_6_2,predict_6_3,predict_6_4,predict_6_5,predict_7_1,predict_7_2,predict_7_3,predict_8_1,predict_8_2,predict_8_3,predict_9_1,predict_9_2,predict_9_3,predict_9_4,predict_9_5,predict_9_6,predict_9_7,predict_9_8,predict_10_1
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1750001,-6.928531,-6.33072,-4.604047,-4.332741,-6.918528,-4.66216,-4.915423,-7.500583,-4.715835,-6.640036,-5.824955,-8.341255,-12.819076,-2.406226,-4.442045,-7.582067,-8.373887,-9.577566,-4.268574,-5.467973,-6.241443,-5.710987,-5.654677,-5.574775,-6.353785,-8.725774,-2.789667,-4.500851,-5.520776,-6.09348,-3.923785,-3.675133,-5.381188,-2.626255,-3.895903,-8.207819,-6.253912,-0.500078,-2.286177,-6.150739,-0.498483
1750002,-5.212724,-5.510101,-3.18002,-4.051201,-6.761028,-3.426863,-4.620086,-7.228794,-4.414074,-5.883624,-6.262423,-8.047687,-13.991214,-2.650402,-5.039015,-7.017035,-7.936301,-9.204812,-6.605817,-4.984972,-6.488889,-5.417718,-5.049805,-5.960865,-5.943228,-8.973015,-3.166242,-4.139491,-5.981587,-4.470534,-2.48613,-4.898022,-6.251198,-1.267649,-3.525211,-4.009518,-4.130578,-0.718358,-1.987024,-8.397829,-1.131405
1750003,-5.549562,-5.569159,-4.044561,-3.276414,-6.591024,-5.54841,-5.551536,-7.847321,-5.347166,-6.326354,-5.382605,-10.284917,-16.017017,-2.26323,-4.321121,-7.585054,-7.769879,-10.078516,-3.691358,-5.281967,-6.999967,-4.962929,-5.92573,-5.928427,-5.588191,-8.952788,-5.856121,-4.425495,-5.938095,-3.72643,-3.482952,-4.293971,-5.503377,-3.510869,-4.390942,-9.037441,-7.441411,-1.320508,-3.310003,-7.026916,-0.595101
1750004,-7.594952,-6.63566,-5.735698,-5.080783,-9.24239,-5.283117,-5.505198,-7.857042,-4.52406,-7.64873,-6.237172,-9.69699,-14.862511,-2.78275,-4.386976,-7.380786,-8.317619,-9.694301,-6.290978,-5.726462,-6.70731,-5.618042,-5.417733,-5.570696,-6.625319,-10.07075,-2.760924,-4.432539,-5.826955,-6.037653,-4.756296,-3.79735,-6.490259,-3.329884,-3.747324,-7.305758,-5.467099,-0.534861,-2.021774,-5.965953,-0.519389
1750005,-4.509488,-5.866231,-3.475819,-3.509484,-8.346423,-4.962579,-5.654517,-6.90916,-5.646854,-5.905872,-5.672499,-10.104343,-15.64425,-2.111308,-4.907119,-6.97267,-8.461734,-9.847859,-6.821068,-5.796301,-6.434449,-4.65988,-4.64246,-6.914616,-6.316195,-9.454053,-6.177919,-4.735089,-6.658656,-5.247823,-3.242404,-4.789762,-6.028573,-2.765586,-3.652401,-8.125484,-4.593967,-1.246197,-2.16552,-6.417112,-0.724723
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1999996,-5.319863,-6.836165,-5.795009,-4.937525,-8.020615,-7.952994,-6.494458,-9.104302,-4.9239,-6.970297,-5.264845,-12.065701,-16.430234,-3.037751,-4.802828,-7.886857,-8.732921,-9.66837,-7.306258,-5.824976,-6.809601,-4.915778,-5.054389,-5.526528,-5.955274,-10.189803,-3.323383,-4.275666,-6.525299,-5.29482,-3.528344,-5.908802,-8.225961,-6.460832,-4.257102,-5.864414,-6.725191,-2.217716,-5.46883,-7.754865,1.08093
1999997,-6.104312,-6.280447,-4.911639,-4.78418,-6.842799,-4.210375,-5.906954,-4.320183,-6.715268,-6.986725,-6.850328,-6.858303,-9.098973,-1.723248,-0.853754,-7.091396,-6.013986,-7.729012,-4.77492,-4.039744,-7.652977,-4.668294,-5.495784,-5.392345,-3.912868,-5.790535,-2.805039,-5.456292,-6.879085,-0.936105,-3.848139,-6.105997,-6.761993,-3.632006,-5.772607,-7.821044,-7.318713,-2.700016,-3.254908,-4.703728,-1.895754
1999998,-6.842299,-8.05102,-7.532999,-6.151014,-8.763651,-6.880423,-8.099641,-7.747337,-5.01656,-6.495952,-5.36115,-11.618969,-16.859552,-2.483372,-0.314186,-8.34373,-7.557464,-4.628049,-6.806089,-5.043848,-6.522388,-4.155875,-6.28835,-5.333801,-3.705964,-8.002547,-4.229154,-6.443822,-6.504887,1.622052,-5.454239,-4.833917,-6.899759,-5.9

In [15]:
catboost_predictions.write_parquet("submit0004.parquet")